<a href="https://colab.research.google.com/github/osgeokr/GEE-PAM-Book/blob/main/Google_Earth_Engine_Sentinel-1_%EC%95%8C%EA%B3%A0%EB%A6%AC%EC%A6%98_%EC%86%8C%EA%B0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Google Earth Engine: Sentinel-1 알고리즘 소개

Sentinel-1은 유럽연합(EU: European Union)이 자금을 지원하고 유럽우주국(ESA: European Space Agency)이 코페르니쿠스 프로그램(Copernicus Programme) 내에서 수행하는 우주 미션입니다. Sentinel-1은 다양한 편파와 해상도에서 C-밴드 합성 개구 레이더(SAR: Synthetic Aperture Radar) 이미지를 수집합니다. 레이더 데이터는 보정된, 정사영상 이미지를 얻기 위해 여러 전문 알고리즘이 필요하기 때문에 Earth Engine에서 Sentinel-1 데이터의 사전 처리를 설명합니다.

Sentinel-1 데이터는 다양한 기기 구성, 해상도, 대역 조합으로 상승 궤도와 하강 궤도 모두에서 수집됩니다. 이러한 이질성 때문에 처리를 시작하기 전에 데이터를 동질의 부분집합으로 필터링하는 것이 보통 필요합니다.

In [1]:
!pip install -q ipyleaflet==0.18

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.2 MB/s eta 0:00:00


In [2]:
import ipyleaflet
print(ipyleaflet.__version__)

0.18.0


In [3]:
import ee
import geemap
import geopandas as gpd
import requests

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-foss4g')

## 메타데이터와 필터링

Sentinel-1 데이터의 동질 부분집합을 만들기 위해서는 보통 메타데이터 속성을 사용하여 컬렉션을 필터링할 필요가 있습니다. 필터링에 사용되는 일반적인 메타데이터 필드에는 다음과 같은 속성들이 포함됩니다:

1. `transmitterReceiverPolarisation`: ['VV'], ['HH'], ['VV', 'VH'], 또는 ['HH', 'HV']
2. `instrumentMode`: 'IW' (Interferometric Wide Swath), 'EW' (Extra Wide Swath) 또는 'SM' (Strip Map).
3. `orbitProperties_pass`: 'ASCENDING' 또는 'DESCENDING'
4. `resolution_meters`: 10, 25 또는 40
5. `resolution`: 'M' (medium) 또는 'H' (high).

다음 코드는 `transmitterReceiverPolarisation`, `instrumentMode`, 및 `orbitProperties_pass` 속성에 따라 Sentinel-1 컬렉션을 필터링하고, 여러 관측 조합에 대한 복합 이미지를 계산한 후 이러한 특성이 데이터에 어떻게 영향을 미치는지 보여주기 위해 지도에 표시합니다.

In [4]:
# Sentinel-1 이미지 컬렉션을 불러오고, 2023년 6-9월까지의 관측 데이터로 필터링
sentinel_1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterDate(
    '2023-06-01', '2023-10-01'
)

In [5]:
# 메타데이터 속성에 따라 Sentinel-1 컬렉션 필터링
vv_vh_iw = (
    sentinel_1
    # VV와 VH 이중 편파로 촬영된 이미지를 얻기 위해 필터링
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
    # IW 모드로 수집된 이미지를 얻기 위해 필터링
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
)

In [6]:
# 상승 궤도와 하강 궤도 이미지를 별도의 컬렉션으로 분리
vv_vh_iw_asc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
)
vv_vh_iw_desc = vv_vh_iw.filter(
    ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
)

In [7]:
# 시각화를 위해 다양한 관측의 시간 평균 계산
# 상승 궤도 VH 평균
vh_iw_asc_mean = vv_vh_iw_asc.select('VH').mean()
# 하강 궤도 VH 평균
vh_iw_desc_mean = vv_vh_iw_desc.select('VH').mean()
# 상승 및 하강 궤도 이미지 컬렉션을 결합한 VV 평균.
vv_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VV').mean()
# 상승 및 하강 궤도 이미지 컬렉션을 결합한 VH 평균.
vh_iw_asc_desc_mean = vv_vh_iw_asc.merge(vv_vh_iw_desc).select('VH').mean()

  > `"wb"`는 'write binary'의 줄임말로, 파일을 쓰기 위해 바이너리 모드를 사용하겠다는 의미입니다. 파일은 크게 텍스트 모드와 바이너리 모드로 두 가지 방식으로 다룰 수 있으며, 바이너리 모드는 파일을 파이트 형태로, 즉 파일의 원래 형식과 정확한 내용을 변경 없이 저장하는 방식을 말합니다.

In [11]:
# 진주시 센서스경계 다운로드
url = "https://github.com/osgeokr/GEE-PAM-Book/raw/main/JINJU.gpkg"
response = requests.get(url)
with open("JINJU.gpkg", "wb") as file:
    file.write(response.content)

# 진주시 센서스경계 읽기
gdf = gpd.read_file("JINJU.gpkg")

# 좌표계를 WGS84 (EPSG:4326)로 변환
gdf_wgs84 = gdf.to_crs(epsg=4326)
gdf_wgs84.head()

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,진주시,38030,"MULTIPOLYGON (((128.15659 35.34964, 128.15689 ..."


In [12]:
# gdf의 중심 좌표를 계산
centroid = gdf_wgs84.geometry.unary_union.centroid
longitude, latitude = centroid.x, centroid.y
print("Longitude:", longitude, "Latitude:", latitude)

Longitude: 128.12976291238812 Latitude: 35.20514831522156


In [17]:
# 다양한 관측의 시간 평균을 표시 및 비교
m = geemap.Map(width="800px", height="500px")
m.add_layer(vv_iw_asc_desc_mean, {'min': -12, 'max': -4}, 'vv_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_desc_mean')
m.add_layer(vh_iw_asc_mean, {'min': -18, 'max': -10}, 'vh_iw_asc_mean')
m.add_layer(vh_iw_desc_mean, {'min': -18, 'max': -10}, 'vh_iw_desc_mean')
m.set_center(longitude, latitude, 12)  # 보고타, 콜롬비아
m

Map(center=[35.20514831522156, 128.12976291238812], controls=(WidgetControl(options=['position', 'transparent_…

## Sentinel-1 전처리

Earth Engine의 `COPERNICUS/S1_GRD` Sentinel-1 `ImageCollection`에 포함된 이미지는 Level-1 Ground Range Detected (GRD), 즉 레벨-1 지상 범위 감지 장면으로 처리되어 후방산란 계수(σ°)를 데시벨(dB) 단위로 나타냅니다. 후방산란 계수는 단위 지면 면적당 목표 후방산란 면적(레이더 단면)을 나타냅니다. 이 값은 수십 배의 차이가 날 수 있으므로, 10*log10σ°의 형식으로 dB 단위로 변환됩니다. 이는 조사된 지형이 입사 마이크로파 복사를 SAR 센서에서 멀어지는 방향으로 (dB < 0) 또는 SAR 센서 쪽으로 (dB > 0) 선호적으로 산란시키는지를 측정합니다. 이러한 산란 행위는 주로 지형 요소의 기하학적 특성과 그들의 전자기적 특성에 따라 달라집니다.

Earth Engine은 각 픽셀에서 후방산란 계수를 도출하기 위해 Sentinel-1 Toolbox에서 구현된 다음과 같은 전처리 단계를 사용합니다:

1. **Apply orbit file**(궤도 파일 적용): 복원된 궤도 파일(또는 복원된 파일이 사용 불가능한 경우 정밀 궤도 파일)로 궤도 메타데이터를 업데이트합니다.

1. **GRD border noise removal**(GRD 테두리 잡음 제거): 장면 가장자리의 저강도 잡음과 유효하지 않은 데이터를 제거합니다. (2018년 1월 12일 기준)

1. **Thermal noise removal**(열 잡음 제거): 다중-스워스 획득 모드에서 장면의 서브-스워스 간 불연속성을 줄이기 위해 서브-스워스의 가산 잡음을 제거합니다. (이 작업은 2015년 7월 이전에 생산된 이미지에는 적용할 수 없습니다)

1. **Application of radiometric calibration values**(방사 보정 값 적용): GRD 메타데이터의 센서 보정 매개변수를 사용하여 후방산란 강도를 계산합니다.

1. **Terrain correction**(지형 보정) 또는 Orthorectification(정사 보정): 지형을 고려하지 않는 지상 범위 기하학에서 SRTM 30미터 DEM 또는 고위도(60° 이상 또는 -60° 이하)의 경우 ASTER DEM을 사용하여 σ°로 데이터를 변환합니다.

##데이터셋 주의사항

- 산지경사에서 발생하는 오류 때문에 Radiometric Terrain Flattening(방사보정 지형 평탄화)는 적용되지 않습니다.

- 단위 없는 후방산란 계수는 위에서 설명한 대로 dB로 변환됩니다.

- Sentinel-1 SLC 데이터는 현재 Earth Engine에서 취급할 수 없습니다. 이는 Earth Engine이 피라미드화하는 동안 위상 정보의 손실 없이 복소수 값을 평균화할 수 없기 때문에 복소수 값을 포함하는 이미지를 지원하지 않기 때문입니다.

- GRD SM 자원은 S1 Toolbox의 테두리 잡음 제거 작업에서 computeNoiseScalingFactor() 함수가 SM 모드를 지원하지 않기 때문에 취급되지 않습니다.